### Поиск на основе регулярных выражений

In [16]:
# переформатирования выводимой строки
fields = ['asdf', ' ', 'fjdk', ';', 'afed', ',', 'fjek', ',', 'asdf', ',', 'foo']
delimeters = fields[1::2] + ['']
values = fields[::2]
''.join([v+d for v,d in zip(values,delimeters)])

'asdf fjdk;afed,fjek,asdf,foo'

In [17]:
# Вы хотите найти текст, используя те же маски, которые обычно используются в оболочках Unix (например, \*.py, Dat[0-9]\*.csv и т.д.)
from fnmatch import fnmatch, fnmatchcase
print(fnmatch('Dat45.csv', 'Dat[0-9]*'))
print(fnmatch('foo.txt', '?oo.txt'))


True
True


In [18]:
addresses = [
    '5412 N CLARK ST',
    '1060 W ADDISON ST',
    '1039 W GRANVILLE AVE',
    '2122 N CLARK ST',
    '4802 N BROADWAY',
]
[addr for addr in addresses if fnmatchcase(addr, '54[0-9][0-9] *CLARK*')]

['5412 N CLARK ST']

In [19]:
# Если вы собираетесь много раз искать по одному и тому же шаблону, часто окупается предварительная компиляция шаблона регулярного выражения в объект шаблона.
import re
date = '2021-12-03'
form = re.compile(r'\d+-\d+-\d+')
if form.match(date):
    print('yes')
else:
    print('no')


yes


In [34]:
# При составлении регулярных выражений часто нужно использовать захватывающие группы, заключая части шаблона в скобки.
form = re.compile(r'(\d+)-(\d+)-(\d+)')
g = form.match(date)
print(g.groups())
yead, month, day = g.groups()
print(yead, month, day)

# Если вы хотите искать совпадения итеративно, используйте метод *finditer()*
for m in form.finditer(date):
    print('iterated', m.groups())

('2021', '12', '03')
2021 12 03
iterated ('2021', '12', '03')


### Поиск и замена текста

In [35]:
# перезаписать даты, чтобы перевести из их формата “11/27/2012” в “2012-11-27.”
text = 'Today is 11/27/2012. PyCon starts 3/13/2013.'
re.sub(r'(\d+)/(\d+)/(\d+)', r'\3-\1-\2', text)

'Today is 2012-11-27. PyCon starts 2013-3-13.'

In [39]:
# более сложных подстановок можно определить подстановочную функцию обратного вызова (коллбэк, callback)
from calendar import month_abbr

form = re.compile(r'(\d+)/(\d+)/(\d+)')

def change_date(m):
    mon_name = month_abbr[int(m.group(1))]
    return '{} {} {}'.format(m.group(2), mon_name, m.group(3))

form.sub(change_date, text)

'Today is 27 Nov 2012. PyCon starts 13 Mar 2013.'

In [42]:
# текст замены не будет совпадать по регистру с заменяемым текстом

text = 'UPPER PYTHON, lower python, Mixed Python'

def matchcase(word):
    def replace(m):
        text = m.group()
        if text.isupper():
            return word.upper()
        elif text.islower():
            return word.lower()
        elif text[0].isupper():
            return word.capitalize()
        else:
            return word
    return replace

re.sub('python', matchcase('snake'), text, flags=re.IGNORECASE)


'UPPER SNAKE, lower snake, Mixed Snake'

### Определение регулярных выражений для поиска кратчайшего совпадения

In [51]:
# Жадная *
text2 = 'Computer says "no." Phone says "yes."'
str_pat = re.compile(r'\"(.*)\"')
print(str_pat.findall(text2))
# добавим ?
str_pat = re.compile(r'\"(.*?)\"')
print(str_pat.findall(text2))
# поддержка символов новой строки
comment = re.compile(r'/\*((?:.|\n)*?)\*/')
text2 = '''/* this is a
... multiline comment */'''
print(comment.findall(text2))
# comment = re.compile(r'/\*(.*?)\*/', re.DOTALL) 
# re.DOTALL заставляет в регулярном выражении совпадать с любыми символами

['no." Phone says "yes.']
['no.', 'yes.']


[' this is a\n... multiline comment ']

### Приведение текста в Unicode к стандартному представлению (нормализация)

In [59]:
# нормализация
s1 = 'Spicy Jalape\u00f1o'
s2 = 'Spicy Jalapen\u0303o'

print(s1, s2, s1==s2)
import unicodedata

t1 = unicodedata.normalize('NFC', s1)
t2 = unicodedata.normalize('NFC', s2)
print(t1, t2, t1 == t2)
t1 = unicodedata.normalize('NFD', s1)
print(''.join(c for c in t1 if not unicodedata.combining(c)))


Spicy Jalapeño Spicy Jalapeño False
Spicy Jalapeño Spicy Jalapeño True
Spicy Jalapeno


In [70]:
# Арабские цифры
num = re.compile('\d+')
d = num.match('\u0661\u0662\u0663')
d.group(0)

'١٢٣'

In [72]:
# очистка текста
a = 'pýtĥöñ is awesome\n'
b = unicodedata.normalize('NFD', a)
b.encode('ascii', 'ignore').decode('ascii')

'python is awesome\n'

In [73]:
# вы хотите удалить целые диапазоны символов или удалить диакритические знаки. str.translate()
import unicodedata
import sys
cmb_chrs = dict.fromkeys(c for c in range(sys.maxunicode)
    if unicodedata.combining(chr(c)))

b = unicodedata.normalize('NFD', a)
print(b)
print(b.translate(cmb_chrs))

pýtĥöñ is awesome

python is awesome



In [78]:
# Если вы хотите использовать в качестве заполняющего символа не пробел, определите его перед символом выравнивания:
text='Hello World'
print(format(text, '=>20s'))
print(format(text, '*^20s'))


=========Hello World
****Hello World*****


In [79]:
# если вы пишете код, который формирует результат из множества небольших строк, 
# подумайте о том, чтобы оформить его как генератор, используя *yield* для производства фрагментов.

def sample():
    yield 'Is'
    yield 'Chicago'
    yield 'Not'
    yield 'Chicago?'

def combine(source, maxsize):
    parts = []
    size = 0
    for part in source:
        parts.append(part)
        size += len(part)
        if size > maxsize:
            yield ''.join(parts)
            parts = []
            size = 0
    yield ''.join(parts)

for part in combine(sample(), 32768):
    f.write(part)

NameError: name 'f' is not defined

In [84]:
# Стоит отметить, что *vars()* также работает с экземплярами.
s = '{name} has {n} messages.'
class Info:
    def __init__(self, name, n):
        self.n = n
        self.name = name
    
g = Info('Guido', 37)
s.format_map(vars(g))     

'Guido has 37 messages.'

### Разбивка текста на фиксированное количество колонок

In [89]:
s = "Look into my eyes, look into my eyes, the eyes, the eyes, \
the eyes, not around the eyes, don't look around the eyes, \
look into my eyes, you're under."
import textwrap

print(textwrap.fill(s, 20, subsequent_indent=' '))

Look into my eyes,
 look into my eyes,
 the eyes, the eyes,
 the eyes, not
 around the eyes,
 don't look around
 the eyes, look into
 my eyes, you're
 under.


In [92]:
import os
os.get_terminal_size().columns

127

In [98]:
# Работа с HTML- и XML-сущностями в тексте
s = 'Spicy &quot;Jalape&#241;o&quot.'
from html.parser import HTMLParser
p = HTMLParser()
print(p.unescape(s))
s = 'Elements are written as "<tag>text</tag>".'
import html
print(html.escape(s))

Spicy "Jalapeño".
Elements are written as &quot;&lt;tag&gt;text&lt;/tag&gt;&quot;.


/Users/a19028558/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  """


In [100]:
# У вас есть строка, которую вы хотите распарсить в поток токенов слева направо.
text = 'foo = 23 + 42 * 10'

NAME = r'(?P<NAME>[a-zA-Z_][a-zA-Z_0-9]*)'
NUM  = r'(?P<NUM>\d+)'
PLUS = r'(?P<PLUS>\+)'
TIMES = r'(?P<TIMES>\*)'
EQ  = r'(?P<EQ>=)'
WS  = r'(?P<WS>\s+)'

master_pat = re.compile('|'.join([NAME, NUM, PLUS, TIMES, EQ, WS]))

from collections import namedtuple

Token = namedtuple('Token', ['type','value'])

def generate_tokens(pat, text):
    scanner = pat.scanner(text)
    for m in iter(scanner.match, None):
        yield Token(m.lastgroup, m.group())

for tok in generate_tokens(master_pat, 'foo = 42'):
    print(tok)

Token(type='NAME', value='foo')
Token(type='WS', value=' ')
Token(type='EQ', value='=')
Token(type='WS', value=' ')
Token(type='NUM', value='42')


In [102]:
# Вы можете просто применить к байтовым строкам поиск совпадений с помощью регулярных выражений, но сами шаблоны должны быть определены как байты.
data = b'FOO:BAR,SPAM'

re.split(b'[:,]',data)

[b'FOO', b'BAR', b'SPAM']